### This method is currently not in use becuase it rebuilds the graph for every transit; this is quite inefficient, and leads to steadily increasing memory usage that looks like a memory leak but isn't. Using theano shared variables is not as simple as it looks at first glance, because the PyMC3 model expects the data vectors to have the same shape every time the model is called.

In [ ]:
single_tts  = []

for npl, p in enumerate(planets):
    print("\nPLANET", npl)
    
    single_tts.append([])
    
    for i, t0 in enumerate(p.tts[p.quality*~p.overlap]):
        
        print(i, np.round(t0,2))

        # first grab the photometry data
        if sc is not None:
            use_sc = np.abs(sc.time - t0) < 1.5*p.duration

        if lc is not None:
            use_lc = np.abs(lc.time - t0) < 1.5*p.duration


        if (np.sum(use_sc) > 0)*(np.sum(use_lc) > 0):
            raise ValueError("Double counting data in both short and long cadence")

        elif np.sum(use_sc) > 0:
            xtime = sc.time[use_sc]
            yflux = sc.flux[use_sc]
            dtype = "short"

        elif np.sum(use_lc) > 0:
            xtime = lc.time[use_lc]
            yflux = lc.flux[use_lc]
            dtype = "long"

        else:
            raise ValueError("Expected either short or long cadence data")


        # next set up the model
        with pm.Model() as single_model:
            # single phyisical parameter (epoch)
            T0 = pm.Normal("T0", mu=t0, sd=p.duration/3)

            # nuissance parameters
            flux0 = pm.Normal("flux0", mu=1.0, sd=np.std(yflux))
            logvar = pm.Normal("logvar", mu=np.log(np.var(yflux)), sd=np.log(4))


            # set up stellar model and planetary orbit
            exoSLC = exo.StarryLightCurve(u)
            orbit  = exo.orbits.KeplerianOrbit(period=periods[npl], t0=T0, b=b[npl], 
                                               r_star=Rstar, m_star=Mstar)

            # set oversampling factor
            if dtype == 'short':
                oversample = 1
            elif dtype == 'long':
                oversample = 15

            # calculate light curves
            light_curves = exoSLC.get_light_curve(orbit=orbit, r=radii[npl], t=xtime, oversample=oversample)
            summed_light_curve = pm.math.sum(light_curves, axis=-1) + flux0*T.ones(len(xtime))
            model_flux = pm.Deterministic('model_flux', summed_light_curve)


            # here's the likelihood
            obs = pm.Normal("obs", mu=model_flux, sd=T.sqrt(T.exp(logvar)), observed=yflux)


        with single_model:
            single_trace = pm.sample(tune=1000, draws=500, start=single_model.test_point, chains=2, 
                                     step=exo.get_dense_nuts_step(target_accept=0.65))


        single_tts[npl].append((np.median(single_trace["T0"]), np.std(single_trace["T0"])))

In [ ]:
single_inds = []

for npl, p in enumerate(planets):
    single_inds.append([])
    for i, ind in enumerate(p.index[p.quality*~p.overlap]):
        single_inds[npl].append(ind)
        
        
for npl in range(NPL):
    single_tts[npl] = np.asarray(single_tts[npl])

In [ ]:
indep_transit_times = []
indep_ephemeris = []
full_indep_ephemeris = []

fig, axes = plt.subplots(NPL, figsize=(12,8))

for npl, p in enumerate(planets):
    inds  = single_inds[npl]
    xtime = single_tts[npl][:,0]
    xerr  = single_tts[npl][:,1]
    ephem = np.polyval(np.polyfit(inds, xtime, 1, w=1/xerr), inds)
    yomc  = (xtime - ephem)*24*60
    
    indep_transit_times.append(xtime)
    indep_ephemeris.append(ephem)
    full_indep_ephemeris.append(np.polyval(np.polyfit(inds, xtime, 1, w=1/xerr), p.index))
    
    
    axes[npl].errorbar(xtime, yomc, yerr=xerr*24*60, fmt='.', color='C{0}'.format(npl))
    axes[npl].set_ylabel('O-C [min]', fontsize=20)
axes[NPL-1].set_xlabel('Time [BJKD]', fontsize=20)
axes[0].set_title(TARGET, fontsize=20)
plt.savefig(FIGURE_DIR + TARGET + '_ttvs_quick.pdf', bbox_inches='tight')
plt.show()